# 문장생성

### 모델 불러오기


In [16]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2"
)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

### 사전학습된 토크나이저 불러오기

In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 그리디서치

In [17]:
import torch
def getSearch(input_ids):
    with torch.no_grad():
      generated_ids = model.generate(
            input_ids,
            do_sample=True,          # False 이면 컨텍스트가 동일하면 결과값이 항상 같으
            min_length=0,
            max_length=300,
            no_repeat_ngram_size=3, # 토큰이 3개 이상 반복될 경우 3번째 토큰 확률을 0으로 
            #repetition_penalty=1.5,
            top_k=50,
            temperature=0.5
        )
    return generated_ids  

###  인코더, 디코더

In [21]:

def getEncoder(str):
    return tokenizer.encode(str, return_tensors="pt")

def getDecoder(generated_ids):
    return tokenizer.decode([el.item() for el in generated_ids[0]]);


###  샘플

In [23]:
encode = getEncoder("안녕하세요")
print(encode)

search = getSearch(encode)
print(search)

decode = getDecoder(search)
print(decode)

tensor([[25906,  8702,  7801,  8084]])
tensor([[25906,  8702,  7801,  8084,   406, 16563,   377,  6947,  7401,   387,
         45887,   739,  7570, 11649,  9078,  7892,  9193,  6824,  8084,   406,
          9050,  6969, 24117,  7281,  7480, 15842,  9265,  7162,   739,  7570,
          9676, 22386,  9193,  6824, 25856, 16563,   377,  7965, 11597,   739,
          7570, 13023, 44227, 42138,  8006, 25856, 16563,  8146,  8196,  7991,
         28315, 14309,  9258, 25518, 35557,  7209,  9639, 13525,   377,  7965,
          6969, 24117, 36625, 15842,  9265,  7470, 11382, 20111,   739,  7570,
          9337,  9025,  9341,  8084,   406,   739,  7570,  9277, 10137, 34019,
          6969,  8084,   406, 16518,  9873,  6824,  9277, 10578,  9098,  8239,
          9025,  9846,  6969,  8084,   376, 16563,   377,  7098,   387,  9181,
          6872, 16691, 16563,   377,  7970,  6872, 16691, 18519,   387,  9265,
          7162,  9265,  7470,  9207, 45887,  9337,  9025,  9080, 10578,  9098,
          765